Documentation:

This part creates a dataset for generating the candidates of users request,
it takes:[{column:query}, {coulumn, query}], and will fetch the top 5 results


the column list accepted here is :

* 1.rate             
* 2.approx_cost  
the numerial variable can take one number, two numbers, or a number and a direction

for example: [
  {'rate',(3,4)},{'approx_cost',200}]

----------------

* 3.location         
* 4.rest_type        
* 5.dish_liked      
* 6.cuisines         
* 7.menu_item        
* 8.listed_type    

pass the variable and the query:

for example
[{'cuisines': 'Italian'},
{'location': 'Banashankari'}]



## Data proprecessing


In [3]:
import pandas as pd
import numpy as np

In [5]:
file_path ='/Users/jamie/Downloads/zomato.csv'
zomato = pd.read_csv(file_path)

In [6]:
zomato.rename(columns={'approx_cost(for two people)':'approx_cost','listed_in(type)':'listed_type','listed_in(city)':'listed_city'},inplace=True)

In [7]:
#1. process the data type for float variables :convert str to float

# Define a lambda function to remove the '/5' from the rate column and convert to float
remove_slash = lambda x: x.replace('/5', '') if isinstance(x, str) else x

# Apply the lambda function to the rate column
# Ensure the column is treated as string before calling .replace()
zomato['rate'] = zomato['rate'].astype(str).apply(remove_slash).str.strip()

# Now, safely convert the rate column to float
zomato['rate'] = pd.to_numeric(zomato['rate'], errors='coerce')


#convert cost into float

#zomato['approx_cost'] = zomato['approx_cost'].astype(str).replace(',', '').astype(float, errors='ignore')
zomato['votes'] = zomato['votes'].astype(str).str.strip().astype(float, errors='ignore')
zomato['approx_cost'] = zomato['approx_cost'].astype(str)
zomato['approx_cost'] = zomato['approx_cost'].apply(lambda x: x.replace(',','.')) #Using lambda function to replace ',' from cost
zomato['approx_cost'] = zomato['approx_cost'].astype(float)


In [8]:
#2. calculated a weighted rating score based on voting

# Calculate C as the mean rating across all restaurants
C = zomato['rate'].mean()

# Set m to a value you choose (for example, the 90th percentile of the votes)
m = zomato['votes'].quantile(0.90)

# Define a function to compute the weighted rating for each row
def weighted_rating(x, m, C):
    v = x['votes']
    R = x['rate']
    return (v/(v+m) * R) + (m/(m+v) * C)

# Apply the function to your DataFrame
zomato['weighted_rating'] = zomato.apply(weighted_rating, axis=1, args=(m, C))

In [9]:
zomato.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost,reviews_list,menu_item,listed_type,listed_city,weighted_rating
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1,775.0,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800.0,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari,3.909956
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1,787.0,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800.0,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari,3.911487
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8,918.0,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800.0,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari,3.756826
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7,88.0,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300.0,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari,3.700399
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8,166.0,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600.0,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari,3.719465


In [10]:

zomato.drop_duplicates(subset=['name','address'],inplace=True)

In [14]:
zomato.drop(['url','address','phone'],axis=1,inplace=True)

In [11]:
#zomato.to_csv('/content/drive/My Drive/Colab Notebooks/data/zomato_ready.csv', index=False)

In [12]:
zomato['weighted_rating'].sort_values(ascending=False)[0:5]

3921     4.850535
8260     4.780858
40506    4.739724
3762     4.699950
39546    4.691678
Name: weighted_rating, dtype: float64

In [13]:
# Assuming you want to display the 'name', 'location', and 'cuisine' columns as well
sorted_zomato = zomato.sort_values('weighted_rating', ascending=False).drop_duplicates(subset=['name'])
sorted_zomato.head(5)

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost,reviews_list,menu_item,listed_type,listed_city,weighted_rating
3921,https://www.zomato.com/bangalore/byg-brewski-b...,"Behind MK Retail, Sarjapur Road, Bangalore",Byg Brewski Brewing Company,Yes,Yes,4.9,16345.0,+91 8039514766,Sarjapur Road,Microbrewery,"Cocktails, Dahi Kebab, Rajma Chawal, Butter Ch...","Continental, North Indian, Italian, South Indi...",1.6,"[('Rated 5.0', 'RATED\n I have been to this p...",[],Delivery,Bellandur,4.850535
8260,https://www.zomato.com/bangalore/abs-absolute-...,"100 Feet Road, 1st Phase, Near Jayadeva Flyove...",AB's - Absolute Barbecues,No,Yes,4.9,6375.0,040 45659912,BTM,Casual Dining,"Tangdi Chicken, Bbq Buffet, Chocolate Icecream...","European, Mediterranean, North Indian, BBQ",1.6,"[('Rated 5.0', 'RATED\n We liked the place a ...",[],Buffet,BTM,4.780858
3762,https://www.zomato.com/bangalore/the-black-pea...,"20/7, Swamy Legato, Outer Ring Road, Kadubeesa...",The Black Pearl,No,Yes,4.8,7023.0,080 49653069,Marathahalli,"Casual Dining, Bar","Dahipuri, Jal-jeera, Chicken Grill, Mutton See...","North Indian, European, Mediterranean, BBQ",1.5,"[('Rated 5.0', 'RATED\n One of the best place...",[],Buffet,Bellandur,4.699950
39546,https://www.zomato.com/bangalore/chilis-americ...,"2nd Floor, Orion Mall, Dr. Rajkumar Road, Mall...",Chili's American Grill & Bar,Yes,Yes,4.8,6433.0,+91 9500075479\n+91 9535541956,Malleshwaram,"Casual Dining, Bar","Veg Platter, Burgers, Mushroom Soup, Cocktails...","American, Tex-Mex, Burger, BBQ, Mexican",1.4,"[('Rated 2.0', ""RATED\n Pricey menu for the q...","['Bacon Ranch Chicken Quesadillas', 'Onion Rin...",Delivery,Malleshwaram,4.691678
41533,https://www.zomato.com/bangalore/the-boozy-gri...,"20/7, 4th & 5th Floor, Swamy Legato, Outer Rin...",The Boozy Griffin,No,Yes,4.8,4696.0,080 45128669,Marathahalli,Pub,"Singapore Sling, Sea Food Platter, Cocktails, ...","European, Continental",1.8,"[('Rated 4.0', 'RATED\n Best calamari i have ...",[],Dine-out,Marathahalli,4.656828


In [14]:
zomato[['name','reviews_list']]

,name,reviews_list
0,Jalsa,"[('Rated 4.0', 'RATED\n A beautiful place to ..."
1,Spice Elephant,"[('Rated 4.0', 'RATED\n Had been here for din..."
2,San Churro Cafe,"[('Rated 3.0', ""RATED\n Ambience is not that ..."
3,Addhuri Udupi Bhojana,"[('Rated 4.0', ""RATED\n Great food and proper..."
4,Grand Village,"[('Rated 4.0', 'RATED\n Very good restaurant ..."
...,...,...
51627,New Shalimar Hotel,[]
51628,Nook - Aloft Bengaluru Cessna Business Park,"[('Rated 2.0', 'RATED\n Aloft has been design..."
51632,SeeYa Restaurant,"[('Rated 4.0', 'RATED\n Good food, take bit t..."
51677,Bhagini,"[('Rated 4.0', 'RATED\n A fine place to chill..."


In [15]:
zomato.columns

Index(['url', 'address', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'phone', 'location', 'rest_type', 'dish_liked', 'cuisines',
       'approx_cost', 'reviews_list', 'menu_item', 'listed_type',
       'listed_city', 'weighted_rating'],
      dtype='object')

## Candidates generation using label

In [22]:
def match_and_filter(df, column, search_term, top_n=5):
    # Perform a precise match first
    precise_matches = df[df[column].astype(str).str.lower() == str(search_term).lower()]

    # If precise match results are fewer than top_n, perform fuzzy match
    if len(precise_matches) < top_n:
        matches = process.extract(search_term, df[column], scorer=process.scorer.WRatio, limit=None)
        matched_indices = [match[2] for match in matches if match[1] > 80]
        fuzzy_matches = df.iloc[matched_indices]
        combined_matches = pd.concat([precise_matches, fuzzy_matches]).drop_duplicates()
    else:
        combined_matches = precise_matches

    # Sort by weighed rating and remove duplicates by name
    top_items = combined_matches.sort_values(by='weighted_rating', ascending=False).drop_duplicates(subset='name').head(top_n)

    return top_items


## Candidates generation based on numerical information

In [23]:
def filter_by_float_column(df, column, value, top_n=5):
    if isinstance(value, (int, float)):
        # Single number, filter within ±20%
        tolerance = 0.20 * value
        lower_bound = value - tolerance
        upper_bound = value + tolerance
        condition = df[column].between(lower_bound, upper_bound)
    elif isinstance(value, (list, tuple)) and len(value) == 2:
        # Two numbers, range filter
        lower_bound, upper_bound = sorted(value)  # Ensure lower to upper order
        condition = df[column].between(lower_bound, upper_bound)
    elif isinstance(value, dict):
        # One number with direction, e.g. {'bigger': 1000}
        number = next(iter(value.values()))
        direction = next(iter(value.keys()))
        if direction == 'bigger':
            condition = df[column] > number
        elif direction == 'smaller':
            condition = df[column] < number
        else:
            raise ValueError("Direction can only be 'bigger' or 'smaller'.")
    else:
        raise ValueError("Value must be a number, a range (list/tuple), or a dict with direction.")

    # Apply the filter
    filtered_df = df[condition]

    # Sort by 'weighed_rating' and remove duplicates by name
    top_items = filtered_df.sort_values(by='weighted_rating', ascending=False).drop_duplicates(subset='name').head(top_n)

    return top_items

##Merge and rank results

In [24]:
def breakdown_and_collect(df, filter_list):
    scores = pd.Series(0, index=df.index)  # Initialize the scores series to zero for all indices

    for filter_dict in filter_list:
        for column, search_term in filter_dict.items():
            if isinstance(search_term, str):
                # Check for precise match in string columns and score 1 if matched
                scores[df[column].astype(str).str.lower() == search_term.lower()] += 1
            elif isinstance(search_term, (int, float)):
                # Check for match within range for float columns and assign appropriate score
                tolerance = search_term * 0.2
                upper_tolerance = search_term * 0.5
                within_tolerance = df[column].between(search_term - tolerance, search_term + tolerance)
                within_upper_tolerance = df[column].between(search_term - upper_tolerance, search_term + upper_tolerance)

                # Score 1 for values within 20% tolerance
                scores[within_tolerance] += 1
                # Score 0.5 for values that exceed 20% but are within 50% tolerance
                scores[(~within_tolerance) & within_upper_tolerance] += 0.5

    df['score'] = scores  # Assign scores to the dataframe
    df_matched = df[df['score'] > 0]  # Filter out rows with zero score

    # Sort by score and weighted_rating, and get top 5 unique restaurants
    df_matched = df_matched.sort_values(by=['score', 'weighted_rating'], ascending=[False, False])
    df_matched_unique = df_matched.drop_duplicates(subset='name').head(5)

    return df_matched_unique.drop(columns=['score'])  # Return results without the 'score' column



In [25]:
# Example filter_list with unique columns
filter_list = [
    {'cuisines': 'Italian'},
    {'location': 'Banashankari'},
    {'approx_cost': 400},  # Exact value with 20% tolerance
]

# Assuming `zomato` is the DataFrame loaded with your data
final_results = breakdown_and_collect(zomato, filter_list)
print(final_results[['name', 'cuisines', 'approx_cost', 'location', 'weighted_rating']])

                       name                                         cuisines  \
52   Corner House Ice Cream                              Ice Cream, Desserts   
54             Roving Feast  Italian, Continental, Fast Food, Chinese, Momos   
145            Anna Kuteera              South Indian, North Indian, Chinese   
191      Mini Punjabi Dhaba                                     North Indian   
13               Caf-Eleven                                Cafe, Continental   

     approx_cost      location  weighted_rating  
52         400.0  Banashankari         3.897820  
54         450.0  Banashankari         3.879666  
145        450.0  Banashankari         3.857134  
191        350.0  Banashankari         3.845268  
13         450.0  Banashankari         3.813146  


In [33]:
import pickle
# Load the model from the file
with open('ensemble_model.pkl', 'rb') as file:
    loaded_ensemble = pickle.load(file)

In [34]:
import spacy

# Load a pre-trained NLP model from spaCy
nlp = spacy.load("en_core_web_sm")

def extract_entities(review_text):
    # Convert non-string data to string
    if not isinstance(review_text, str):
        review_text = str(review_text)
    doc = nlp(review_text)
    return [(ent.text, ent.label_) for ent in doc.ents]

In [37]:
import numpy as np
def rank_restaurants(model_pipeline, restaurants, sentence):
    # Extract entities from the sentence
    entities = extract_entities(sentence)
    
    # Combine the restaurant names with the entities
    combined_features = [f"{restaurant} {entities}" for restaurant in restaurants]
    
    # Initialize an empty list to hold probabilities
    probabilities = [] 
    # Iterate through each fitted classifier within the VotingClassifier
    for name, clf in model_pipeline.named_estimators_.items():
        # Check if the named classifier is a pipeline
        if hasattr(clf, 'named_steps'):
            # Transform features using the fitted vectorizer in the pipeline
            features_vectorized = clf.named_steps['vectorizer'].transform(combined_features)
            
            # Predict probabilities using the trained model in the pipeline
            clf_probabilities = clf.named_steps['classifier'].predict_proba(features_vectorized)[:, 1]
            
            # Append probabilities to the list
            probabilities.append(clf_probabilities)
    
    # Average the probabilities from all classifiers
    average_probabilities = np.mean(probabilities, axis=0)
    
    # Create a DataFrame with the results
    ranking_df = pd.DataFrame({
        'Restaurant': restaurants,
        'Entities': len(restaurants) * [entities],
        'Probability_of_Recommendation': average_probabilities
    })
    
    # Sort the DataFrame by probability of recommendation
    ranking_df.sort_values(by='Probability_of_Recommendation', ascending=False, inplace=True)
    
    return ranking_df

In [38]:
restaurant_list= list(final_results['name'])
sentence = "recommend a italian pub"               
ranked_df = rank_restaurants(loaded_ensemble, restaurant_list, sentence)

In [41]:
def combine_rank_with_info(ranked_df, info_df):
    # Merge the ranked DataFrame with the info DataFrame based on the restaurant name
    combined_df = pd.merge(ranked_df, info_df, left_on='Restaurant', right_on='name', how='left')
    
    # Select and rename the columns you want to include in the final DataFrame
    final_df = combined_df[['Restaurant', 'Entities', 'Probability_of_Recommendation', 'phone', 'location', 'approx_cost']]
    
    return final_df

combined_info_df = combine_rank_with_info(ranked_df, final_results)

print(combined_info_df)


               Restaurant           Entities  Probability_of_Recommendation  \
0  Corner House Ice Cream  [(italian, NORP)]                       0.825126   
1      Mini Punjabi Dhaba  [(italian, NORP)]                       0.687004   
2              Caf-Eleven  [(italian, NORP)]                       0.603197   
3            Roving Feast  [(italian, NORP)]                       0.577250   
4            Anna Kuteera  [(italian, NORP)]                       0.508558   

                              phone      location  approx_cost  
0    080 26713965\r\n+91 9845444155  Banashankari        400.0  
1  +91 9845044074\r\n+91 7406012344  Banashankari        350.0  
2                      080 49577715  Banashankari        450.0  
3      080 42024092\r\n080 42274242  Banashankari        450.0  
4                    +91 9620009302  Banashankari        450.0  


# create a classifation model

In [ ]:
# First, ensure that the 'Rating' column is numeric and drop any non-numeric rows
resto_reviews['Rating'] = pd.to_numeric(resto_reviews['Rating'], errors='coerce')
resto_reviews = resto_reviews.dropna(subset=['Rating'])

# Next, remove rows with no rating (if any still remain)
resto_reviews = resto_reviews[resto_reviews['Rating'].notnull()]

# Now, apply the lambda function to create the 'Recommendation' column
resto_reviews['Recommendation'] = resto_reviews['Rating'].apply(lambda x: 'recommend' if x >= 3.0 else 'no recommend')

# The updated DataFrame now has a 'Recommendation' column with values based on the 'Rating' column


In [ ]:
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

resto_reviews["Review"] = resto_reviews["Review"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

resto_reviews["Review"] = resto_reviews["Review"].apply(lambda text: remove_urls(text))